<a href="https://colab.research.google.com/github/mounikamarreddy/BERTFinetuning-NLPTasks/blob/main/BERTEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.8 MB/s 
     |████████████████████████████████| 6.6 MB 34.5 MB/s 
     |████████████████████████████████| 120 kB 58.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch

In [5]:
from transformers import BertModel, BertTokenizer

In [43]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name, output_hidden_states = True,output_attentions=True)
input_text = "I am learning transformers with brother help"

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [44]:
# tokenise the text
tokenized_texts = [tokenizer.tokenize(input_text)]
print ("Tokenize the sentence:")
print (tokenized_texts[0])

Tokenize the sentence:
['i', 'am', 'learning', 'transformers', 'with', 'brother', 'help']


In [45]:
# tokenize the sentence and generate input_id's
input_ids = tokenizer.encode(input_text, add_special_tokens=True)

In [46]:
#CLS token at start and SEP token at end are added
input_ids

[101, 1045, 2572, 4083, 19081, 2007, 2567, 2393, 102]

In [47]:
input_ids = torch.tensor([input_ids])

In [48]:
with torch.no_grad():
    outputs = model(input_ids)
    attentions = outputs.attentions

In [49]:
# the length of outputs represents last_hidden_state, pooler_output, hidden_states, attentions
# last_hidden_state (batch_size, sequence_length, hidden_size) : last hidden state which is outputted from the last BertLayer
# pooler_output (batch_size, hidden_size) : output of the Pooler layer
# hidden_states (batch_size, sequence_length, hidden_size): hidden-states of the model at the output of each BertLayer plus the initial embedding
# attentions (batch_size, num_heads, sequence_length, sequence_length): one for each BertLayer. Attentions weights after the attention SoftMax
len(outputs)

4

In [55]:
print(outputs[0].shape)

torch.Size([1, 9, 768])


In [51]:
len(attentions)

12

In [29]:
hidden_states = outputs[2]

In [40]:
len(hidden_states)

13

In [32]:
# layer_0 is for initial embeddings, 12 layers are for 12 hidden layer representations
layer_0 = hidden_states[0]

In [33]:
# batch_size = 1, tokens = 9, embedding dimension = 768
layer_0.shape

torch.Size([1, 9, 768])

In [34]:
layer_0

tensor([[[ 1.6855e-01, -2.8577e-01, -3.2613e-01,  ..., -2.7571e-02,
           3.8253e-02,  1.6400e-01],
         [-3.4027e-04,  5.3974e-01, -2.8805e-01,  ...,  7.5731e-01,
           8.9008e-01,  1.6575e-01],
         [-6.3496e-01,  1.9748e-01,  2.5116e-01,  ..., -4.0819e-02,
           1.3468e+00, -6.9357e-01],
         ...,
         [ 2.7237e-01, -8.8593e-01, -7.5636e-01,  ..., -8.7015e-01,
          -2.1279e-02,  5.4287e-01],
         [-8.1437e-01,  3.1857e-03,  1.0250e-01,  ...,  2.7126e-01,
           2.8317e-01,  5.3041e-01],
         [-2.3061e-01, -2.4164e-01, -4.0893e-02,  ..., -2.8555e-01,
           2.4370e-01, -7.0539e-02]]])

In [41]:
#bert layer_1 representations
layer_1 = hidden_states[1]

In [42]:
layer_1.shape

torch.Size([1, 9, 768])

In [52]:
# attention representations, index 0 represents layer 1 attentions and index 11 represents layer 12 attentions.
layer_1_attention = attentions[0]

In [53]:
layer_1_attention.shape

torch.Size([1, 12, 9, 9])

In [54]:
layer_1_attention

tensor([[[[7.2432e-02, 6.1376e-02, 5.1153e-02, 6.1012e-02, 6.2050e-02,
           1.8046e-01, 8.6079e-02, 6.9843e-02, 3.5560e-01],
          [1.9949e-01, 1.4013e-01, 7.6084e-02, 9.3578e-02, 1.7148e-01,
           5.0118e-02, 1.0816e-01, 6.4796e-02, 9.6175e-02],
          [6.3968e-02, 4.3015e-01, 4.2197e-02, 9.5562e-02, 1.1792e-01,
           3.5626e-02, 5.5822e-02, 4.4204e-02, 1.1455e-01],
          [8.1623e-02, 1.9603e-01, 6.7857e-02, 7.1092e-02, 1.7276e-01,
           6.3544e-02, 1.4040e-01, 1.2922e-01, 7.7480e-02],
          [4.0270e-02, 1.9172e-01, 1.0148e-01, 9.5134e-02, 1.3129e-01,
           6.0334e-02, 6.7838e-02, 1.4467e-01, 1.6725e-01],
          [1.8078e-01, 8.7074e-02, 7.0630e-02, 6.2005e-02, 9.5623e-02,
           1.8117e-01, 7.8268e-02, 1.3857e-01, 1.0588e-01],
          [8.6552e-02, 2.1255e-01, 9.9102e-02, 1.3601e-01, 1.8765e-01,
           6.2388e-02, 5.9054e-02, 9.0893e-02, 6.5799e-02],
          [1.6474e-01, 1.6625e-01, 1.1314e-01, 6.2576e-02, 1.0309e-01,
           1